In [3]:
import json
import os
from unskript import nbparams
from unskript.secrets import ENV_MODE, ENV_MODE_AWS
from unskript.fwk.workflow import Task, Workflow

env = {"ENV_MODE":"ENV_MODE_AWS","TENANT_ID":"982dba5f-d9df-48ae-a5bf-ec1fc94d4882","PROXY_ID":"1499f27c-6406-4fbd-bd1b-c6f92800018f","TENANT_URL":"https://tenant-staging.alpha.unskript.io","AWS_REGION":"us-west-2"}
secret_store_cfg = {"SECRET_STORE_TYPE":"SECRET_STORE_TYPE_AWS","AWS_SECRET_PREFIX":"staging","AWS_REGION":"us-west-2"}
os.environ["UNSKRIPT_REDIS_HOST"] = "redis-master.redis.svc.cluster.local"
os.environ["UNSKRIPT_TOKEN"] = "5c4a5754-0600-11ec-9a03-0242ac130003"
os.environ["TENANT_URL"] = env["TENANT_URL"]
paramDict = {"channel": None, "interval": "5"}
paramDict.update(env)
paramDict.update(secret_store_cfg)
paramsJson = json.dumps(paramDict)
nbParamsObj = nbparams.NBParams(paramsJson)
channel = nbParamsObj.get('channel')
interval = nbParamsObj.get('interval')

w = Workflow(env, secret_store_cfg, None, global_vars=globals(), check_uuids=None)


<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to get PostgreSQL long running queries using unSkript legos.</b>
</div>

<br>

<center><h2>Display PostgreSQL Long Running Queries</h2></center>

# Steps Overview
    1) Get postgres long running queries by passing the interval as input
    2) Collecting PostgreSQL queries and post that to the slack channel

<p>Here we will use unSkript Long Running PostgreSQL Queries Action.</p>
<p>This action takes&nbsp;<strong>interval</strong>: <strong>int</strong> as input. This input is used to find out all the Long running queries available on the PostgreSQL database.</p>
<p>It returns the list of PIDs that exceed given interval.</p>

In [7]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from typing import List, Any, Union

from tabulate import tabulate
from pydantic import BaseModel, Field


from beartype import beartype
def legoPrinter(func):
    def Printer(*args, **kwargs):
        output = func(*args, **kwargs)
        if len(output) == 0:
            print("No queries found")
            return

        print("\n")
        print(output)
        return output

    return Printer


@legoPrinter
@beartype
def postgresql_long_running_queries(handle, interval: int = 5) -> Union[List[Any], str]:
    """postgresql_long_running_queries Runs postgres query with the provided parameters.

          :type credentialsDict: dict
          :param credentialsDict: Dictionary of credentials info.

          :type inputParamsJson: string
          :param inputParamsJson: Json string of the input params

          :rtype: All the results of the query.
      """
    # Input param validation.

    # Multi-line will create an issue when we package the Legos.
    # Hence concatinating it into a single line.

    query = "SELECT pid, user, pg_stat_activity.query_start, now() - pg_stat_activity.query_start AS query_time, query, state " \
        " FROM pg_stat_activity WHERE state = 'active' AND (now() - pg_stat_activity.query_start) > interval '%d seconds';" % interval

    cur = handle.cursor()
    cur.execute(query)
    output = []
    res = cur.fetchall()

    data = []
    for records in res:
        result = {
            "pid": records[0],
            "user": records[1],
            "query_start": records[2],
            "query_time": records[3],
            "query": records[4],
            "state": records[5]
        }
        output.append(result)
        data.append([records[0], records[4], records[5], records[3]])

    if len(res) > 0:
        headers = ["pid", "query", "state", "duration"]
        print("\n")
        output = tabulate(data, headers=headers, tablefmt="grid")

    handle.commit()
    cur.close()
    handle.close()
    return output


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "segment",
    "credential_type": "CONNECTOR_TYPE_POSTGRESQL",
    "credential_id": "05050797-8929-47de-aaed-93d9fc63b244"
}''')
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "interval": "5"
    }''')
task.configure(outputName="sql_queries")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(postgresql_long_running_queries, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

Here we will use unSkript Post Slack Message Lego. This lego takes channel: str and message: str as input. This inputs is used to post the message to the slack channel.

In [9]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
def legoPrinter(func):
    def Printer(*args, **kwargs):
        output = func(*args, **kwargs)
        if output:
            channel = kwargs["channel"]
            pp.pprint(print(f"Message sent to Slack channel {channel}"))
        return output
    return Printer


@legoPrinter
@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> bool:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return True
    except SlackApiError as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        return False
    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return False


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "unskript_community",
    "credential_type": "CONNECTOR_TYPE_SLACK",
    "credential_id": "c81e186c-0a93-4b12-a0db-4eaa185efa6a"
}''')
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "channel": "channel",
    "message": "f\\"Long Running Queries : {sql_queries}\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "channel is not None and sql_queries is not None",
    "condition_result": true
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(slack_post_message, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

<h3 id="Conclusion">Conclusion<a class="jp-InternalAnchorLink" href="#Conclusion" target="_self">&para;</a></h3>
<p>In this Runbook, we demonstrated the use of unSkript's PostgreSQL action to run PostgreSQL Query, determines the long running queries from a database and (optionally) send a message to Slack. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://unskript.com" target="_blank" rel="noopener">https://unskript.com</a></p>